*** UNDER CONSTRUCTION ***


# HTLC-Timeout and HTLC-Success Transactions
## Prerequisite knowledge
### For all notebooks
- A high level understanding of the bitcoin e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook), in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
- A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
- [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).
- A high level understanding of the lightning e.g. [Mastering Lightning Network](https://github.com/lnbook/lnbook), in particular [Chapter7](https://github.com/lnbook/lnbook/blob/develop/07_payment_channels.asciidoc), [Chapter 8](https://github.com/lnbook/lnbook/blob/develop/08_routing_htlcs.asciidoc) and [Chapter 9](https://github.com/lnbook/lnbook/blob/develop/09_channel_operation.asciidoc).

### Specific to this notebook
- SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
- Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
- Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
- Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'
- Lightning Network BOLT #3: '[Commitment Transactions](https://github.com/lightning/bolts/blob/master/03-transactions.md#commitment-transaction)'
- Finite Fields, Elliptic Curves and Serialization e.g. [Programming Bitcoin](https://github.com/jimmysong/programmingbitcoin), in particular [Chapter 1](https://github.com/jimmysong/programmingbitcoin/blob/master/ch01.asciidoc), [Chapter2](https://github.com/jimmysong/programmingbitcoin/blob/master/ch02.asciidoc), [Chapter 3](https://github.com/jimmysong/programmingbitcoin/blob/master/ch03.asciidoc) and [Chapter 4](https://github.com/jimmysong/programmingbitcoin/blob/master/ch04.asciidoc).

## HTLC-Timeout and HTLC-Success Transactions

The HTLC-Timeout and HTLC-Success Transactions are almost identical, except the HTLC-timeout transaction is timelocked. Both HTLC-timeout/HTLC-success transactions can be spent by a valid penalty transaction.

* version: 2
* locktime: 0 for HTLC-success, cltv_expiry for HTLC-timeout
* txin count: 1
    * txin[0] outpoint: txid of the commitment transaction and output_index of the matching HTLC output for the HTLC transaction
    * txin[0] sequence: 0 (set to 1 for option_anchors)
    * txin[0] script bytes: 0
    * txin[0] witness stack: 0 <remotehtlcsig> <localhtlcsig>  <payment_preimage> for HTLC-success, 0 <remotehtlcsig> <localhtlcsig> <> for HTLC-timeout

* txout count: 1
    * txout[0] amount: the HTLC amount_msat divided by 1000 (rounding down) minus fees in satoshis (see Fee Calculation)
    * txout[0] script: version-0 P2WSH with witness script as shown below

if option_anchors applies to this commitment transaction, SIGHASH_SINGLE|SIGHASH_ANYONECANPAY is used as described in [BOLT #5](https://github.com/lightning/bolts/blob/master/05-onchain.md#generation-of-htlc-transactions).

The witness script for the output is:

```
OP_IF
    # Penalty transaction
    <revocationpubkey>
OP_ELSE
    `to_self_delay`
    OP_CHECKSEQUENCEVERIFY
    OP_DROP
    <local_delayedpubkey>
OP_ENDIF
OP_CHECKSIG
```


## Setup
### Requirements

For this exercise we'll run the previous notebook to create the commitment transactions beetween with offer and received HTLC.

**You'll need to edit these next line for your local setup.**

In [1]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 4 - Unilateral Closing Transaction/Unilateral Closing Transaction.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Revocation Basepoint Private Key: c17ac3952ca414190074d1e59ea03fbae253196173908dc8b131af6bd2cc8161
Alice Revocation Basepoint Public Key: 03649c4f865bec74b0a186deef4defad51cfdc141443e38074ea05a7835a953a49
Alice HTLC Basepoint Private Key: 763ae49a20e6668c88602c782716dd83ba6c4cc0333b38810e2bcd7b22c871ac
Alice HTLC Basepoint Public Key: 02816fde4150e4dfcac94eff0b821448fb70f57a56148ba2206cd9b2fd0cc20bdf
Alice Payment Basepoint Private Key: 72d8c12971b58076a1f27eb7938ca442f0b210762b23637443ac2e99dac352a6
Alice Payment Basepoint Public Key: 025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Alice Delayed Payment Basepoint Private Key: 7cafce00c54e7241894dcc7c3beaca29dd354139fdb6182198d6c5f1063bfe8d
Alice Delayed Payment Basepoint Public Key: 034aa35219136bb238e072341b20a4bf8fb44a83cdb73dd2bd9

### Creating the HTLC-Timeout and HTLC-Success Transactions

Now that the commitment transaction, containing HTLC outputs, was published forcing close the channel, we can create the HTLC-Timeout and/or HTLC-Success Transactions to reclaim the funds locked on HTLCs.

Let's start with simulating an HTLC Timeout transaction.

In [2]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER
marker = bytes.fromhex("00")
# FLAG
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(alice_signed_commitment_txid))[::-1]
index = bytes.fromhex("0200 0000")

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: set to 1 for option_anchors
sequence = bytes.fromhex("0100 0000")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

print("inputs :", inputs.hex())

# OUTPUTS
output_count = bytes.fromhex("01")

# OP_IF
#   Penalty transaction
#   <revocationpubkey>
# OP_ELSE
#    `to_self_delay`
#    OP_CHECKSEQUENCEVERIFY
#    OP_DROP
#    <local_delayedpubkey>
# OP_ENDIF
# OP_CHECKSIG
htlc_redeemScript = bytes.fromhex(
    "63"   # OP_IF
    + "21"
    + bob_revocation_pubkey
    + "67"   # OP_ELSE
    + "02"   
    + "9000" # to_self_delay of 144 blocks
    + "b2"   # OP_CHECKSEQUENCEVERIFY
    + "75"   # OP_DROP
    + "21"
    + alice_delayed_pubkey
    + "68"  # OP_ENDIF
    + "ac") # OP_CHECKSIG

htlc_script_hash = sha256(htlc_redeemScript)
htlc_output_spk = bytes.fromhex("0020") + htlc_script_hash
print(f"HTLC Output SPK: {htlc_output_spk.hex()}")

outputs = (
    alice_offered_htlc_output_value
    + varint_len(htlc_output_spk)
    + htlc_output_spk
)

# LOCKTIME
locktime = height + 5
locktime = locktime.to_bytes(4, 'little')

unsigned_htlc_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

print("unsigned_htlc_tx: ", unsigned_htlc_tx.hex())



inputs : d30080805cdca3e1a314f342c25f075448f7ace3f4374f8b292ff72f333094d6020000000001000000
HTLC Output SPK: 0020978f0c6d76e9a391a84ab6385c34d6cc8dcafdc624523ede9241b8cc2211eb76
unsigned_htlc_tx:  0200000001d30080805cdca3e1a314f342c25f075448f7ace3f4374f8b292ff72f333094d60200000000010000000120a1070000000000220020978f0c6d76e9a391a84ab6385c34d6cc8dcafdc624523ede9241b8cc2211eb766d000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

Just uncomment the lines below to do that.

In [3]:
decoded = node.decoderawtransaction(unsigned_htlc_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "9323be28d92f0ab1b4f299c1e6d13b7479954f70e7e22b20c2290ac5bac356bf",
  "hash": "9323be28d92f0ab1b4f299c1e6d13b7479954f70e7e22b20c2290ac5bac356bf",
  "version": 2,
  "size": 94,
  "vsize": 94,
  "weight": 376,
  "locktime": 109,
  "vin": [
    {
      "txid": "d69430332ff72f298b4f37f4e3acf74854075fc242f314a3e1a3dc5c808000d3",
      "vout": 2,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 1
    }
  ],
  "vout": [
    {
      "value": "0.00500000",
      "n": 0,
      "scriptPubKey": {
        "asm": "0 978f0c6d76e9a391a84ab6385c34d6cc8dcafdc624523ede9241b8cc2211eb76",
        "desc": "addr(bcrt1qj78scmtkax3er2z2kcu9cdxkejxu4lwxy3frah5jgxuvcgs3admqdd0pwu)#j92lqkdq",
        "hex": "0020978f0c6d76e9a391a84ab6385c34d6cc8dcafdc624523ede9241b8cc2211eb76",
        "address": "bcrt1qj78scmtkax3er2z2kcu9cdxkejxu4lwxy3frah5jgxuvcgs3admqdd0pwu",
        "type": "witness_v0_scripthash"
      }
    }
  ]
}


### Signing the HTLC-Timeout and HTLC-Success Transactions

Segwit transactions have a signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki). According to the sighash type (ALL, NONE, SINGLE, ANYONECANPAY), a transaction digest is generated with a double SHA256 of a serialized subset of the transaction (tx_digest_preimage), and the signature is verified against this digest with a given public key.

#### Creating the Transaction Digest Preimage

The sighash type of tis transaction will join two flags:

- SIGHASH_SINGLE (0x03) - The signature only commits to the output with the same index as the input being signed. If the output at that index doesn’t exist, the tx is invalid. This allows transactions with multiple inputs to independently sign for different outputs.

    Example: Alice signs input 0 and only commits to output 0.

- SIGHASH_ANYONECANPAY (0x80) - The signature only commits to the specific input being signed (not other inputs). Anyone else can add inputs to the tx without invalidating this signature.

In [4]:
# Convert txid little endian
txid = bytes.fromhex(alice_signed_commitment_txid)[::-1]
alice_offered_htlc_output_index = 2
# Convert index to bytes (little endian)
index = alice_offered_htlc_output_index.to_bytes(4, byteorder="little", signed=False)
hashPrevOuts = hash256(txid + index)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)

sighash_type = bytes.fromhex("8300 0000") # SIGHASH_SINGLE|SIGHASH_ANYONECANPAY

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + txid
    + index
    + varint_len(alice_offered_htlc_redeemScript)
    + alice_offered_htlc_redeemScript
    + alice_offered_htlc_output_value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print("tx_digest_preimage: ",tx_digest_preimage.hex())

tx_digest_preimage:  0200000091184edd8873e19e6ae5759813856db33e849d244b9d754a32f7b07c78f9fbfe41f758f2e5cc078d3795b4fc0cb60c2d735fa92cc020572bdc982dd2d564d11bd30080805cdca3e1a314f342c25f075448f7ace3f4374f8b292ff72f333094d6020000008876a9145aab2da71aa3e8e99c72c7b07b04fb3a689447d68763ac672103f0ce4fe88d9e1d79c449ced61ee6cfc9280b3e424a48f952667b7fd73a299aa17c820120876475527c2103f020b85b2f600cdba67ecb33ed2ed09bf1c0a0e67e839a9ca70766049a5f329852ae67a914c51b66bced5e4491001bd702669770dccf44098288ac6851b2756820a107000000000001000000c8537014c8acc06799c874db2786837f66e57129eff5f04fa0741ef6eaa722406d00000083000000


## Key Derivations

Each commitment transaction uses a unique localpubkey, local_htlcpubkey, remote_htlcpubkey, local_delayedpubkey and remote_delayedpubkey pubkeys. The derivation of these pubkeys are simply generated by addition from their base points.

The **localpubkey, local_htlcpubkey, remote_htlcpubkey, local_delayedpubkey and remote_delayedpubkey** pubkeys are simply generated by addition from their base points. As defined at Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md#key-derivation)):

```
pubkey = basepoint + SHA256(per_commitment_point || basepoint) * G

```

- The localpubkey uses the local node's payment_basepoint;
- The local_htlcpubkey uses the local node's htlc_basepoint;
- The remote_htlcpubkey uses the remote node's htlc_basepoint;
- The local_delayedpubkey uses the local node's delayed_payment_basepoint;
- The remote_delayedpubkey uses the remote node's delayed_payment_basepoint.


The **revocationpubkey** is a blinded key, when the local node wishes to create a new commitment for the remote node, it uses its own `revocation_basepoint` and the remote node's `per_commitment_point` to derive a new `revocationpubkey` for the commitment. After the remote node reveals the `per_commitment_secret` used (thereby revoking that commitment), the local node can then derive the `revocationprivkey`, as it now knows the two secrets necessary to derive the key (`revocation_basepoint_secret` and `per_commitment_secret`). As defined at Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md#revocationpubkey-derivation)):

```
revocationpubkey = revocation_basepoint * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_point * SHA256(per_commitment_point || revocation_basepoint)
```

This construction ensures that neither the node providing the `basepoint` nor the node providing the `per_commitment_point` can know the private key without the other node's secret.

The corresponding private key can be derived once the `per_commitment_secret` is known:

```
revocationprivkey = revocation_basepoint_secret * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_secret * SHA256(per_commitment_point || revocation_basepoint)
```

We already derived the pubkeys when we created the commitment transaction. Now let's derive the corresponding private key to sign the HTLC-timeout transaction.

In [5]:
# ALICE HTLC PRIVKEY
# alice_htlc_privkey = alice_basepoint_secret + SHA256(per_commitment_point || basepoint)
# alice_htlc_basepoint_secret is generate by alice
# per_commitment_point is generated by alice and sent into the `revoke_and_ack` message
# Compute SHA256(per_commitment_point || basepoint)
sha_output = sha256(bytes.fromhex(alice_per_commitment_point) + bytes.fromhex(alice_htlc_basepoint_pubkey))
sha_int = int.from_bytes(sha_output, 'big')
# do the math to get the alice_htlc_privkey
tweaked_priv_int = (int.from_bytes(bytes.fromhex(alice_htlc_basepoint_privkey), 'big') + sha_int) % n
alice_htlc_privkey = tweaked_priv_int.to_bytes(32, 'big')
print(f"Alice HTLC Privkey: {alice_htlc_privkey.hex() }")

# BOB HTLC PRIVKEY
# bob_htlc_privkey = bob_basepoint_secret + SHA256(per_commitment_point || basepoint)
# bob_htlc_basepoint_secret is generate by bob
# per_commitment_point is generated by bob and sent into the `revoke_and_ack` message
# Compute SHA256(per_commitment_point || basepoint)
sha_output = sha256(bytes.fromhex(bob_per_commitment_point) + bytes.fromhex(bob_htlc_basepoint_pubkey))
sha_int = int.from_bytes(sha_output, 'big')
# do the math to get the alice_htlc_privkey
tweaked_priv_int = (int.from_bytes(bytes.fromhex(bob_htlc_basepoint_privkey), 'big') + sha_int) % n
bob_htlc_privkey = tweaked_priv_int.to_bytes(32, 'big')
print(f"Bob HTLC Privkey: {bob_htlc_privkey.hex() }")

Alice HTLC Privkey: 4538248364791b00f39654406f0ef3102c4184e2da8a51815f976abc368e07ea
Bob HTLC Privkey: 9c6f51fffe0edb251d33ae9a3b7385b735f8de728765c10a640dc14e7c3d2ad4


In [6]:
# Create the Transaction Digest (sigHash) to be signed
sighash = hash256(tx_digest_preimage)

# Sign the sigHash with the alice htlc private key
alice_htlc_signing_key = ecdsa.SigningKey.from_string(alice_htlc_privkey, curve=ecdsa.SECP256k1) 
alice_htlc_signature = alice_htlc_signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append # SIGHASH_SINGLE|SIGHASH_ANYONECANPAY to the alice signature
alice_htlc_signature = alice_htlc_signature + bytes.fromhex("83")

# Sign the sigHash with the bob htlc private key
bob_htlc_signing_key = ecdsa.SigningKey.from_string(bob_htlc_privkey, curve=ecdsa.SECP256k1) 
bob_htlc_signature = bob_htlc_signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append # SIGHASH_SINGLE|SIGHASH_ANYONECANPAY to the bob signature
bob_htlc_signature = bob_htlc_signature + bytes.fromhex("83")

# Witness field
# 0 <remotehtlcsig> <localhtlcsig> <> for HTLC-timeout
witness = (
    bytes.fromhex("05")                  # indicate the number of stack items
    + bytes.fromhex("00")                # Add an extra "00" for the CheckMultiSig bug
    + varint_len(bob_htlc_signature)
    + bob_htlc_signature
    + varint_len(alice_htlc_signature)
    + alice_htlc_signature
    + bytes.fromhex("00") 
    + varint_len(alice_offered_htlc_redeemScript)
    + alice_offered_htlc_redeemScript
)

# the final signed transaction
signed_htlc_timeout_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs # scriptsig left empty as espcified on BIP
    + output_count
    + outputs
    + witness
    + locktime
)

print("HTLC Timeout Transaction: ",signed_htlc_timeout_tx.hex())

HTLC Timeout Transaction:  02000000000101d30080805cdca3e1a314f342c25f075448f7ace3f4374f8b292ff72f333094d60200000000010000000120a1070000000000220020978f0c6d76e9a391a84ab6385c34d6cc8dcafdc624523ede9241b8cc2211eb76050047304402207b1f2a51809dd3f8214f912e4a237ed2ec8e9155904fa9196b3ec42891fe169402203996493a795f4629593f4f15e58e22b60662202349f422584f4a381f3bb031c283483045022100e5f4d78b1ae99e7f4f2cde4d7172f4dfe965f57defc372a8bff6b1f0851790f002206c4fe8689f467c4043013f33fc1e57261728afb76e17843dbad8fa8533185fcd83008876a9145aab2da71aa3e8e99c72c7b07b04fb3a689447d68763ac672103f0ce4fe88d9e1d79c449ced61ee6cfc9280b3e424a48f952667b7fd73a299aa17c820120876475527c2103f020b85b2f600cdba67ecb33ed2ed09bf1c0a0e67e839a9ca70766049a5f329852ae67a914c51b66bced5e4491001bd702669770dccf44098288ac6851b275686d000000


In [10]:
result = node.testmempoolaccept(rawtxs=[signed_htlc_timeout_tx.hex()])
print(result)

[{'txid': '9323be28d92f0ab1b4f299c1e6d13b7479954f70e7e22b20c2290ac5bac356bf', 'wtxid': '8fc5c71c8b6d772997b313465fc50dea3a3d7711b436e0f525bdfae59ebfd59d', 'allowed': False, 'reject-reason': 'non-final'}]


The rejected reason (non-final) indicate that the transaction has to wait the block height defined on locktime. So let's make it hapen mining 5 blocks.

In [11]:
mining_address = node.getnewaddress()
txid = node.generatetoaddress(nblocks=5, address=mining_address, invalid_call=False)

In [12]:
result = node.testmempoolaccept(rawtxs=[signed_htlc_timeout_tx.hex()])
print(result)

[{'txid': '9323be28d92f0ab1b4f299c1e6d13b7479954f70e7e22b20c2290ac5bac356bf', 'wtxid': '8fc5c71c8b6d772997b313465fc50dea3a3d7711b436e0f525bdfae59ebfd59d', 'allowed': False, 'reject-reason': 'min relay fee not met'}]


Now the reject reason has changed to "min relay fee not met". This is expected, as the transaction was created to receive additional inputs that will cover the fees. This behavior is handled by the Sweeper Subsystem of Lightning nodes (see the specific notebook for details).